# Holocaust Memorial Dashboard #

This code creates an interactive web dashboard that visualizes the locations of Holocaust concentration camps and their subcamps across Europe. Here's a detailed breakdown of its structure and functionality:

## Data Handling and Preparation ##

The code begins by importing necessary libraries:
- `pandas` and `numpy` for data processing
- `dash` framework components for creating the web application
- `plotly.express` for interactive map visualization

It loads camp location data from a CSV file named `Camps.csv` and adds a dictionary mapping main camps to estimated Jewish death counts. This data is used to create scaled marker sizes (using a logarithmic scale) to visually represent the magnitude of deaths at each **MAIN camp**.

Additional contextual information about each main camp is stored in the `camp_info` dictionary, which provides brief historical descriptions that will be displayed when users interact with the map.

## Dashboard Structure ##

The dashboard is built using `Dash`, a Python framework for building analytical web applications. It consists of:
1. **Header Section:**
    - A title "Holocaust Subcamp Dashboard"
    - A subtitle explaining marker sizing
2. **Control Panel (left side, 25% width):**
    - Main camp filter dropdown (multi-select with "All" option)
    - Map style selector with 3 options (OpenStreetMap, Carto Light, Carto Dark)
    - Information box that displays details when a marker is clicked
3. **Map Display (right side, 75% width):**
    - Interactive map using Plotly's `scatter_mapbox`
    - Camps displayed as colored dots sized according to death counts
    - Centered on Europe with appropriate zoom level
4. **Footer:**
    - Links to authoritative data sources including Auschwitz Museum, USHMM, Holocaust Encyclopedia, and Yad Vashem

## Interactive Features ##

The dashboard includes two callback functions that enable interaction:
1. `update_map()` - Triggered when filter selections change:
    - Filters the dataset based on selected main camps
    - Updates the map with filtered data
    - Configures hover information and custom data for each point
2. `display_info()` - Triggered when a marker is clicked:
    - Extracts data about the selected camp
    - Formats and displays detailed information in the info box
    - Shows subcamp name, main camp affiliation, death count, and historical context

## Technical Implementation ##
- The map uses **Mapbox** integration through `Plotly`
- Marker sizes are **logarithmically** scaled to handle the wide range of death counts
- Custom data is attached to each point to support the information display
- The layout is responsive with flexible sizing

In [2]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

### 1. Load and prepare data ###

In [4]:
df = pd.read_excel('Camps.xlsx')
df.head()

,MAIN,SUBCAMP,LAT,LONG
0,Auschwitz,Altdorf,49.983299,18.916700
1,Auschwitz,Althammer,51.400001,17.466699
2,Auschwitz,Auschwitz I,50.026199,19.204099
3,Auschwitz,Auschwitz II Birkenau,50.037100,19.175200
4,Auschwitz,Auschwitz III Monowitz,50.031101,19.291500


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MAIN     952 non-null    object 
 1   SUBCAMP  952 non-null    object 
 2   LAT      952 non-null    float64
 3   LONG     952 non-null    float64
dtypes: float64(2), object(2)
memory usage: 29.9+ KB


In [6]:
# Estimated Jewish death counts per main camp
murder_dict = {
    'Auschwitz': 1100000,
    'Bergen-Belsen': 70000,
    'Buchenwald': 11800,
    'Dachau': 41500,
    'Flossenbürg': 30000,
    'Herzogenbusch': 749,
    'Hinzert': 1000,
    'Krakau-Plaszów': 8000,
    'Lublin': 78000,
    'Neuengamme': 42900,
    'Natzweiler-Struthof': 22000
}

In [7]:
# Map death counts and compute marker sizes (log scale)
df['MurderedJews'] = df['MAIN'].map(murder_dict)
# Compute marker sizes (log scale, minimum for zero)
min_size = 5
scale = 10
def size_func(x): return min_size if x == 0 else np.log10(x + 1) * scale
df['MarkerSize'] = df['MurderedJews'].apply(size_func)

df

,MAIN,SUBCAMP,LAT,LONG,MurderedJews,MarkerSize
0,Auschwitz,Altdorf,49.983299,18.916700,1100000.0,60.413931
1,Auschwitz,Althammer,51.400001,17.466699,1100000.0,60.413931
2,Auschwitz,Auschwitz I,50.026199,19.204099,1100000.0,60.413931
3,Auschwitz,Auschwitz II Birkenau,50.037100,19.175200,1100000.0,60.413931
4,Auschwitz,Auschwitz III Monowitz,50.031101,19.291500,1100000.0,60.413931
...,...,...,...,...,...,...
947,Neuengamme,"Porta Westfalica/A II Barkhausen [aka Porta I,...",52.246341,8.912079,42900.0,46.324674
948,Neuengamme,Porta Westfalica/A II Lerbeck-Neesen,52.249500,8.943290,42900.0,46.324674
949,Neuengamme,"Porta Westfalica/A II Hausberge ""Hammerwerke""",52.237294,8.950548,42900.0,46.324674
950,Vaivara,Vaivara subcamp,59.366700,27.750000,NaN,NaN


In [8]:
# Additional camp info for click details
camp_info = {
    'Auschwitz': 'Largest complex: Auschwitz I, II–Birkenau, III–Monowitz.',
    'Bergen-Belsen': 'Initially POW camp, later concentration camp.',
    'Buchenwald': 'One of first and biggest German camps.',
    'Dachau': 'First Nazi camp opened 1933.',
    'Flossenbürg': 'Forced labor in quarries.',
    'Herzogenbusch': 'Only SS camp in occupied Netherlands.',
    'Hinzert': 'Smaller camp in western Germany.',
    'Krakau-Plaszów': 'Camp in occupied Poland.',
    'Lublin': 'Majdanek camp and death site.',
    'Neuengamme': 'Labor camp near Hamburg.',
    'Natzweiler-Struthof': 'Only camp on current French territory.'
}
df['CampInfo'] = df['MAIN'].map(camp_info).fillna('No additional info')

df

,MAIN,SUBCAMP,LAT,LONG,MurderedJews,MarkerSize,CampInfo
0,Auschwitz,Altdorf,49.983299,18.916700,1100000.0,60.413931,"Largest complex: Auschwitz I, II–Birkenau, III..."
1,Auschwitz,Althammer,51.400001,17.466699,1100000.0,60.413931,"Largest complex: Auschwitz I, II–Birkenau, III..."
2,Auschwitz,Auschwitz I,50.026199,19.204099,1100000.0,60.413931,"Largest complex: Auschwitz I, II–Birkenau, III..."
3,Auschwitz,Auschwitz II Birkenau,50.037100,19.175200,1100000.0,60.413931,"Largest complex: Auschwitz I, II–Birkenau, III..."
4,Auschwitz,Auschwitz III Monowitz,50.031101,19.291500,1100000.0,60.413931,"Largest complex: Auschwitz I, II–Birkenau, III..."
...,...,...,...,...,...,...,...
947,Neuengamme,"Porta Westfalica/A II Barkhausen [aka Porta I,...",52.246341,8.912079,42900.0,46.324674,Labor camp near Hamburg.
948,Neuengamme,Porta Westfalica/A II Lerbeck-Neesen,52.249500,8.943290,42900.0,46.324674,Labor camp near Hamburg.
949,Neuengamme,"Porta Westfalica/A II Hausberge ""Hammerwerke""",52.237294,8.950548,42900.0,46.324674,Labor camp near Hamburg.
950,Vaivara,Vaivara subcamp,59.366700,27.750000,NaN,NaN,No additional info


In [9]:
# List of unique main camps and map styles
df_valid = df[df['MurderedJews'] > 0]
main_camps = sorted(df_valid['MAIN'].unique())
options_main = ['All'] + main_camps

map_styles = {
    'OpenStreetMap': 'open-street-map',
    'Carto Light': 'carto-positron',
    'Carto Dark': 'carto-darkmatter'
}

In [10]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 457 entries, 0 to 949
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MAIN          457 non-null    object 
 1   SUBCAMP       457 non-null    object 
 2   LAT           457 non-null    float64
 3   LONG          457 non-null    float64
 4   MurderedJews  457 non-null    float64
 5   MarkerSize    457 non-null    float64
 6   CampInfo      457 non-null    object 
dtypes: float64(4), object(3)
memory usage: 28.6+ KB


### 2. Initialize Dash app ###

In [12]:
app = dash.Dash(__name__, meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}])
app.title = 'Holocaust Subcamp Dashboard'

### 3. App layout ###

In [14]:
app.layout = html.Div([
    html.H1(
        'Holocaust Subcamp Dashboard',
        style={'textAlign': 'center'}
    ),
    html.P(
        'Marker size = log-scale of total Jewish deaths in main camp.',
        style={'textAlign': 'center'}
    ),
    html.Div([
        html.Div([
            html.H4('Filters'),
            html.Label('Main Camps:'),
            dcc.Dropdown(
                id='main-filter',
                options=[{'label': c, 'value': c} for c in options_main],
                value=['All'],
                style={'color': 'black'},
                multi=True
            ),
            html.Br(),
            html.Label('Map Style:'),
            dcc.Dropdown(
                id='style-filter',
                options=[{'label': k, 'value': v} for k, v in map_styles.items()],
                value='open-street-map',
                style={'color': 'black'}
            ),
            html.Br(),
            html.Div(
                id='info-box',
                children='Click a marker for details.',
                style={'whiteSpace': 'pre-line'})
        ], style={'width': '25%', 'float': 'left', 'padding': '10px'}),
        html.Div([dcc.Graph(id='map', style={'height': '80vh'})], style={'width': '75%', 'float': 'right'})
    ], style={'display': 'flex'}),
    html.Footer(html.P(["Data sources: ",
        html.A('Auschwitz Museum', href='https://www.auschwitz.org/en/', target='_blank'), ', ',
        html.A('USHMM', href='https://www.ushmm.org/', target='_blank'), ', ',
        html.A('Holocaust Encyclopedia', href='https://encyclopedia.ushmm.org/', target='_blank'), ', ',
        html.A('Yad Vashem Collections', href='https://collections.yadvashem.org/', target='_blank')
    ]), style={'textAlign': 'center', 'fontSize': 'small', 'marginTop': '20px'})
])

### 4. Callbacks ###

In [16]:
@app.callback(
    Output('map', 'figure'),
    [Input('main-filter', 'value'), Input('style-filter', 'value')]
)
def update_map(selected, style):
    if selected and 'All' not in selected:
        d = df[df['MAIN'].isin(selected) & (df['MurderedJews'] > 0)]
    else:
        d = df[df['MurderedJews'] > 0]
    
    fig = px.scatter_mapbox(
        d,
        lat='LAT',
        lon='LONG',
        hover_name='SUBCAMP',
        hover_data={'MAIN': True, 'MurderedJews': True},
        custom_data=['MAIN','MurderedJews','CampInfo'],
        size='MarkerSize',
        color='MAIN',
        mapbox_style=style,
        zoom=4,
        center={'lat':50,'lon':15},
        title='Holocaust Subcamps: Main Camp Fatalities'
    )
    fig.update_layout(margin=dict(l=0,r=0,t=30,b=0), coloraxis_colorbar=dict(title='Main Camp'))
    
    return fig

In [17]:
@app.callback(
    Output('info-box', 'children'),
    Input('map', 'clickData')
)
def display_info(clickData):
    if not clickData:
        return 'Click a marker for details.'
    pt = clickData['points'][0]
    subcamp = pt['hovertext']
    cd = pt.get('customdata', [])
    main = cd[0] if len(cd)>0 else 'N/A'
    deaths = cd[1] if len(cd)>1 else 'N/A'
    info = cd[2] if len(cd)>2 else 'No additional info'
    return f"Subcamp: {subcamp}\nMain Camp: {main}\nDeaths in main Camp: {int(deaths):,}" if isinstance(deaths,(int,float)) else f"Subcamp: {subcamp}\nMain Camp: {main}\nDeaths: {deaths}\n{info}"

### 5. Run server ###

In [19]:
if __name__ == '__main__':
    app.run_server(debug=True)